# Height Metrics

In [3]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np

data_dir = Path("../data")
plots_dir = data_dir / "outputs" / "plots"
metrics_dir = plots_dir / "metrics"
net_cdf_dir = metrics_dir / "x1-y1-z1" / "net_cdf"
height_metrics_dir = metrics_dir / "height_metrics"

height_metrics_dir.mkdir(parents=True, exist_ok=True)

In [4]:
plots = gpd.read_file(plots_dir / "plots.geojson")
plots = plots.set_index("site_plot_id")
plots

,site,plot_number,id,geometry
site_plot_id,,,,
AGG_O_01_P1,AGG_O_01,1,AGG_O_01_P1,"POLYGON ((463042.83 5259846.736, 463025.797 52..."
AGG_O_01_P2,AGG_O_01,2,AGG_O_01_P2,"POLYGON ((463124.556 5259819.234, 463116.068 5..."
AGG_O_01_P3,AGG_O_01,3,AGG_O_01_P3,"POLYGON ((463201.174 5259815.806, 463200.551 5..."
AGG_O_01_P4,AGG_O_01,4,AGG_O_01_P4,"POLYGON ((463257.777 5259801.962, 463245.303 5..."
AGG_O_01_P5,AGG_O_01,5,AGG_O_01_P5,"POLYGON ((463303.022 5259789.552, 463289.794 5..."
...,...,...,...,...
ULY_Y_96_P1,ULY_Y_96,1,ULY_Y_96_P1,"POLYGON ((455348.476 5275834.517, 455348.154 5..."
ULY_Y_96_P2,ULY_Y_96,2,ULY_Y_96_P2,"POLYGON ((455296.576 5275837.224, 455299.223 5..."
ULY_Y_96_P3,ULY_Y_96,3,ULY_Y_96_P3,"POLYGON ((455252.092 5275834.16, 455248.849 52..."


In [13]:
def read_metrics_ds(plot_id: str) -> xr.Dataset:
    path = net_cdf_dir / f"{plot_id}.nc"
    return xr.open_dataset(path)

def get_height_metrics(row: pd.Series) -> pd.Series:
    plot_id = row.name
    ds = read_metrics_ds(plot_id)
    ds = ds[['max_height', 'mean_height', 'median_height', 'q10_veg_height', 'q20_veg_height', 'q30_veg_height', 'q40_veg_height', 'q50_veg_height', 'q60_veg_height', 'q70_veg_height', 'q80_veg_height', 'q90_veg_height']]
    mean_df = ds.mean().to_pandas().add_prefix('mean_of_') 
    max_df = ds.max().to_pandas().add_prefix('max_of_')
    sd_df = ds.std().to_pandas().add_prefix('sd_of_')
    combined = pd.concat([mean_df, max_df, sd_df])
    
    return combined
    
height_metrics = plots.apply(get_height_metrics, axis='columns')
height_metrics

,mean_of_max_height,mean_of_mean_height,mean_of_median_height,mean_of_q10_veg_height,mean_of_q20_veg_height,mean_of_q30_veg_height,mean_of_q40_veg_height,mean_of_q50_veg_height,mean_of_q60_veg_height,mean_of_q70_veg_height,...,sd_of_median_height,sd_of_q10_veg_height,sd_of_q20_veg_height,sd_of_q30_veg_height,sd_of_q40_veg_height,sd_of_q50_veg_height,sd_of_q60_veg_height,sd_of_q70_veg_height,sd_of_q80_veg_height,sd_of_q90_veg_height
site_plot_id,,,,,,,,,,,,,,,,,,,,,
AGG_O_01_P1,9.080404,4.064385,4.576714,0.826158,2.094260,3.607828,4.964793,5.941776,6.597077,7.135009,...,3.207521,1.436735,2.337780,2.771922,2.886525,2.911822,2.905454,2.907659,2.898932,2.901547
AGG_O_01_P2,8.974677,4.130647,4.733175,1.139634,2.580032,4.114841,5.355336,6.223354,6.835341,7.325921,...,3.218499,1.690006,2.478173,2.724155,2.741152,2.749033,2.754871,2.760767,2.761087,2.788042
AGG_O_01_P3,10.793109,5.021879,5.655639,1.219964,2.737324,4.482400,5.977403,7.103866,7.919373,8.558716,...,3.512059,1.829640,2.690922,3.138900,3.211842,3.161809,3.087424,3.033360,2.970419,2.907859
AGG_O_01_P4,26.233368,12.846590,13.375082,2.973622,5.638581,8.552843,11.551583,14.183249,16.632436,18.893134,...,14.053851,4.465663,7.489648,10.099954,12.401373,14.118188,15.462890,16.621881,17.584189,18.333689
AGG_O_01_P5,33.688303,17.567901,18.189441,6.049558,9.974606,13.307173,16.630384,19.677250,22.621745,25.396015,...,15.119571,7.456581,10.025645,11.927640,13.812487,15.401924,16.741856,17.904603,18.682804,19.363774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ULY_Y_96_P1,8.352560,3.963489,3.828607,1.690977,2.383310,3.008597,3.701521,4.335297,4.995939,5.678679,...,4.185971,2.601882,3.203192,3.596327,3.999796,4.323140,4.633520,4.906046,5.136458,5.307041
ULY_Y_96_P2,6.698925,2.686194,2.467382,0.607797,1.049879,1.630939,2.234598,2.953496,3.576592,4.199027,...,4.413235,1.426247,2.180571,3.175705,3.964581,4.811672,5.466182,5.975558,6.393790,6.721604
ULY_Y_96_P3,4.669076,1.633595,1.292778,0.488729,0.675738,0.897068,1.172479,1.539182,1.944506,2.430607,...,1.938293,0.576046,0.738959,1.082485,1.571437,2.165251,2.649559,3.187197,3.532450,4.019286


In [14]:
height_metrics.to_csv(height_metrics_dir / 'height_metrics.csv')
height_metrics.reset_index().rename(columns={'index':'site_plot_id'}).to_json(height_metrics_dir / 'height_metrics.json', orient='records')

In [12]:
site_info = pd.read_csv(data_dir / "source" / "environmental_data__site_information.csv")
site_info = site_info.drop(columns=['Site_ID', 'Suitability', 'Notes', 'Coupe_code', 'Include/Reserve', 'Label_code', 'PTPZ/WWHA', 'Ownership', 'Road_relative', 'Tree_line_relative', 'Comment Janneke and Sue'])
site_info = site_info.rename(columns={
    "Master_ID": "site_id",
    "Class": "site_type",
    "Estabyear": "year_estab"
})

site_info

,site_id,site_type,Disturbance,Area,Grid,year_estab,Latitude,Longitude,Easting,Northing,elev_mean,slope_mean,Heat_load_index,Geo_Fert_Score,Geo_Fert_Class
0,AGG_O_01,AGG,8,Mid-South,B7,2011.0,-42.813558,146.546391,463112.3521,5259829.825,499.878173,8.674255,0.832902,5,Medium
1,AGG_O_05,AGG,8,Middle,B5,2011.0,-42.592817,146.455115,455305.3571,5284265.081,379.045078,0.638262,0.937101,7,Limestone
2,AGG_O_07,AGG,8,North,C3,2013.0,-42.460205,146.571147,464711.0000,5299090.000,504.282342,11.798216,0.854043,3,High
3,AGG_Y_02,AGG,8,South,E10,2019.0,-43.073428,146.901332,491962.3000,5231052.700,374.493798,10.157180,0.831258,5,Medium
4,AGG_Y_03,AGG,8,South,E12,2019.0,-43.271624,146.886887,490808.0745,5209008.105,280.278210,12.214606,0.855070,3,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,ULY_Y_01,ULY,9,North,B3,2016.0,-42.406104,146.515784,459717.0000,5304858.000,312.855386,9.331517,0.888552,6,Medium
62,ULY_Y_231,ULY,9,Middle,B6,2018.0,-42.735796,146.486496,457973.2000,5268330.800,495.344440,14.730459,0.779000,8,Low
63,ULY_Y_232,ULY,9,South,D11,2019.0,-43.130071,146.787963,482621.0000,5224770.800,152.054739,9.794871,0.836131,6,Medium
64,ULY_Y_25,ULY,9,Mid-South,D9,2019.0,-43.009072,146.745502,479258.9330,5238144.952,262.755022,18.912603,0.874041,5,Medium


In [ ]:
site_info.to_csv(data_dir / "outputs" / "site_info.csv")